In [21]:
from pytorch_metric_learning import miners, losses
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision
from torch.utils.data import DataLoader, random_split
from data_utils import *
from train_utils import *

%load_ext autoreload
%autoreload 2

config = {
    'IMG_WIDTH': 256,
    'IMG_HEIGHT': 256,
    'TRAINING_DATASET_DIR': '../Week 1/data/MIT_split/train',
    'TEST_DATASET_DIR': '../Week 1/data/MIT_split/test',
    'batch_size': 16,
    'epochs': 2,
    'learning_rate': 0.001,
    'device': torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
transform_train = CustomTransform(config, mode='train')
transform_test = CustomTransform(config, mode='test')

train_dataset = PairDataset(datasets.ImageFolder(root=config['TRAINING_DATASET_DIR'], transform=transform_train))
test_dataset =  PairDataset(datasets.ImageFolder(root=config['TEST_DATASET_DIR'], transform=transform_test))

total_length = len(train_dataset)
train_size = int(0.8 * total_length)  # e.g., 80% for training
valid_size = total_length - train_size  # remaining 20% for validation

# Split dataset
train_dataset, validation_dataset = random_split(train_dataset, [train_size, valid_size])

dataloader_train = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
dataloader_validation = DataLoader(validation_dataset, batch_size=config['batch_size'], shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

In [23]:
# Model Definition remains the same
class SiameseNetwork(torch.nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights='ResNet50_Weights.DEFAULT').eval()
        self.model.fc = nn.Identity()

    def forward(self, x1, x2):
        output1 = self.model(x1)
        output2 = self.model(x2)
        return output1, output2
    
    def get_embedding(self, x):
        return self.model(x)

In [25]:
model = SiameseNetwork()
model.to(config['device'])

# Using Contrastive Loss as before
loss_func = losses.ContrastiveLoss(pos_margin=0, neg_margin=1)

# Integrating PairMarginMiner
miner = miners.PairMarginMiner(pos_margin=0.2, neg_margin=0.8)

optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

# Adjusted Training Loop with Miner
for epoch in range(config['epochs']):

    train_loss = train(model, dataloader_train, optimizer, loss_func, miner, config['device'])
    val_loss = validate(model, dataloader_validation, loss_func, miner, config['device'])
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss}, Validation Loss: {val_loss}")

Using cache found in C:\Users\Luis/.cache\torch\hub\pytorch_vision_v0.10.0
  0%|          | 0/94 [00:28<?, ?it/s]

torch.Size([16, 2048]) torch.Size([16, 2048]) torch.Size([16])


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Luis\miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Luis\AppData\Local\Temp\ipykernel_10824\841626648.py", line 15, in <module>
    train_loss = train(model, dataloader_train, optimizer, loss_func, miner, config['device'])
  File "c:\Users\Luis\Documents\Universidad\MCV\C5\MCV-C5\Week 3\train_utils.py", line 17, in train
    hard_pairs = miner(outputs1, outputs2, targets)
  File "c:\Users\Luis\miniconda3\lib\site-packages\torch\nn\modules\module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "c:\Users\Luis\miniconda3\lib\site-packages\torch\nn\modules\module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
  File "c:\Users\Luis\miniconda3\lib\site-packages\pytorch_metric_learning\miners\base_miner.py", line 49, in forward
    c_f.check_shapes(embeddings, labels)
